In [5]:
from modelzipper.tutils import *
from tqdm import trange


CodeLLaMA_PATH = "/zecheng2/model_hub/CodeLlama-7b-hf"
FILE_PATH = "/zecheng2/vqllama/test_vqllama_quantizer/test_0/visualized_compress_level_1/svg_paths.jsonl"
COMPRESSED_PATH = "/zecheng2/vqllama/test_vqllama_quantizer/test_0/compress_level_1_predictions.pkl"

llama_tokenizer = AutoTokenizer.from_pretrained(CodeLLaMA_PATH)
str_cotent = auto_read_data(FILE_PATH)
compress_content = auto_read_data(COMPRESSED_PATH)

# count compress svg tokens
p_predict = compress_content['p_predict']
golden = compress_content['golden']

def count_non_pad_rows(x):
    non_pad_rows = 0
    for row in x:
        row_list = row.tolist()
        if row_list[0] == 0:
            if any(row_list[1:]):
                non_pad_rows += 1
            else:
                break
        else:
            non_pad_rows += 1
    
    return non_pad_rows


def count_svg_tokens(batch_x):
    """
    batch_x: [b, l, 9]
    """
    total_tokens = 0
    for i in trange(len(batch_x)):
        item = batch_x[i]
        non_pad_rows = count_non_pad_rows(item)
        total_tokens += non_pad_rows * 9
    avg_tokens = total_tokens / len(batch_x) 
    return avg_tokens

golden_svg_tokens = count_svg_tokens(golden)
p_predict_svg_tokens = count_svg_tokens(p_predict)

print("golden_svg_tokens: ", golden_svg_tokens)
print("p_predict_svg_tokens: ", p_predict_svg_tokens)


def count_str_tokens(batch_x, tokenizer: AutoTokenizer):
    """
    batch_x: List[str]
    """
    total_tokens = 0
    for i in trange(len(batch_x)):
        item = batch_x[i]
        tokens = tokenizer(item)['input_ids']
        total_tokens += len(tokens)
    avg_tokens = total_tokens / len(batch_x) 
    return avg_tokens


p_svg_str = [item['p_svg_str'] for item in str_cotent]
g_svg_str = [item['g_svg_str'] for item in str_cotent]

p_svg_str_tokens = count_str_tokens(p_svg_str, llama_tokenizer)
g_svg_str_tokens = count_str_tokens(g_svg_str, llama_tokenizer)

print("p_str_tokens: ", p_svg_str_tokens)
print("golden_str_tokens: ", g_svg_str_tokens)


def count_codebook_tokens(batch_x):
    pass



In [15]:
print(compress_content['zs'][0].shape)
print(compress_content['zs'][1].shape)
print(compress_content['zs'][2].shape)

torch.Size([2000, 256])
torch.Size([2000, 128])
torch.Size([2000, 64])
